In [1]:
import pybaseball
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split

In [2]:
batter_info = pybaseball.statcast('2024-04-03', '2024-04-10')
batter_info.head()

This is a large query, it may take a moment to complete


  0%|          | 0/8 [00:00<?, ?it/s]c:\Users\yjain\Desktop\Personal Work\moonshot\.venv\Lib\site-packages\pybaseball\datahelpers\postprocessing.py:59: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  data_copy[column] = data_copy[column].apply(pd.to_datetime, errors='ignore', format=date_format)
 12%|█▎        | 1/8 [00:07<00:51,  7.37s/it]c:\Users\yjain\Desktop\Personal Work\moonshot\.venv\Lib\site-packages\pybaseball\datahelpers\postprocessing.py:59: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  data_copy[column] = data_copy[column].apply(pd.to_datetime, errors='ignore', format=date_format)
 25%|██▌       | 2/8 [00:08<00:20,  3.44s/it]c:\Users\yjain\Desktop\Personal Work\moonshot\.venv\Lib\site-packages\pybaseball\datahelpers\postprocessing.py:59: Future

,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,...,batter_days_until_next_game,api_break_z_with_gravity,api_break_x_arm,api_break_x_batter_in,arm_angle,attack_angle,attack_direction,swing_path_tilt,intercept_ball_minus_batter_pos_x_inches,intercept_ball_minus_batter_pos_y_inches
2498,KC,2024-04-10,82.4,-2.93,4.82,"Kimbrel, Craig",680776,518886,field_out,hit_into_play,...,1,4.18,-0.94,0.94,27.3,-5.721336,24.730087,23.731415,50.254745,10.520772
2622,FF,2024-04-10,92.1,-2.85,4.68,"Kimbrel, Craig",680776,518886,NaN,ball,...,1,1.37,0.76,-0.76,26.2,<NA>,<NA>,<NA>,<NA>,<NA>
2712,FF,2024-04-10,93.4,-2.79,4.69,"Kimbrel, Craig",680776,518886,NaN,ball,...,1,1.47,0.69,-0.69,26.6,<NA>,<NA>,<NA>,<NA>,<NA>
2819,FF,2024-04-10,93.2,-2.9,4.65,"Kimbrel, Craig",680776,518886,NaN,foul,...,1,1.43,0.77,-0.77,25.4,-8.288437,26.934428,25.11755,40.307207,10.417215
2907,FF,2024-04-10,93.0,-2.86,4.6,"Kimbrel, Craig",680776,518886,NaN,foul,...,1,1.26,0.84,-0.84,24.3,3.802008,-0.326281,25.301449,42.623156,26.240147


In [3]:
shortened_data = batter_info[['batter', 'pitch_type', 'description', 'release_speed', 'release_pos_x', 'launch_speed', 'launch_angle', 'effective_speed', 'release_spin_rate', 'release_extension']]
pruned_data = shortened_data.dropna()
print(pruned_data.shape)
pruned_data.head()

(9870, 10)


,batter,pitch_type,description,release_speed,release_pos_x,launch_speed,launch_angle,effective_speed,release_spin_rate,release_extension
2498,680776,KC,hit_into_play,82.4,-2.93,67.8,-45,83.9,2631,7.3
2819,680776,FF,foul,93.2,-2.9,75.0,22,94.5,2417,7.3
2907,680776,FF,foul,93.0,-2.86,73.8,23,94.6,2458,7.3
3115,666152,KC,foul,84.2,-2.99,70.7,-20,85.8,2588,7.3
1669,681297,CU,hit_into_play,80.4,3.21,64.5,-1,79.2,2774,5.7


In [17]:
batter_enc = LabelEncoder()
pruned_data['batter_id'] = batter_enc.fit_transform(pruned_data['batter']) # We may not need this as 'batter' is already a label
pitch_enc = LabelEncoder()
pruned_data['pitch_type_id'] = pitch_enc.fit_transform(pruned_data['pitch_type'])
outcome_enc = LabelEncoder()
pruned_data['outcome_id'] = outcome_enc.fit_transform(pruned_data['description'])

NUM_BATTERS = pruned_data['batter_id'].nunique()
NUM_PITCHES = pruned_data['pitch_type_id'].nunique()
NUM_OUTCOMES = pruned_data['outcome_id'].nunique()

scaler = StandardScaler()
pruned_data[['release_speed', 'release_pos_x', 'launch_speed', 'launch_angle', 'effective_speed', 'release_spin_rate', 'release_extension']] = scaler.fit_transform(pruned_data[['release_speed', 'release_pos_x', 'launch_speed', 'launch_angle', 'effective_speed', 'release_spin_rate', 'release_extension']].astype(float))

len(pruned_data['description'].value_counts())
print(NUM_BATTERS, NUM_PITCHES, NUM_OUTCOMES)


402 14 2


C:\Users\yjain\AppData\Local\Temp\ipykernel_60716\1619357233.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pruned_data['batter_id'] = batter_enc.fit_transform(pruned_data['batter']) # We may not need this as 'batter' is already a label
C:\Users\yjain\AppData\Local\Temp\ipykernel_60716\1619357233.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pruned_data['pitch_type_id'] = pitch_enc.fit_transform(pruned_data['pitch_type'])
C:\Users\yjain\AppData\Local\Temp\ipykernel_60716\1619357233.py:6: SettingW

In [19]:
class SuperDataSet(Dataset):
    def __init__(self, df):
        self.df = df.reset_index(drop=True)
    def __getitem__(self, index):
        row = self.df.loc[index]
        batter_id = int(row['batter_id'])
        pitch_type_id = int(row['pitch_type_id'])
        features = torch.tensor(row[['release_speed', 'release_pos_x', 'launch_speed', 'launch_angle', 'effective_speed', 'release_spin_rate', 'release_extension']].values.astype(np.float32), dtype=torch.float32).squeeze(0)
        y = int(row['outcome_id'])
        return batter_id, pitch_type_id, features, y
    def __len__(self):
        return len(self.df)




train_df, test_df = train_test_split(pruned_data, test_size=0.2, random_state=42, stratify=pruned_data['outcome_id'])

train_dataset = SuperDataSet(train_df)
test_dataset = SuperDataSet(test_df)

print(f"Train dataset size: {len(train_dataset)}")
print(train_dataset[1])
print(f"Test dataset size: {len(test_dataset)}")
print(test_dataset[1])

train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

next(iter(train_loader))


Train dataset size: 7896
(64, 11, tensor([-1.4125, -1.4736, -1.5719, -1.5633, -1.4155,  0.8388,  0.5919]), 1)
Test dataset size: 1974
(227, 6, tensor([ 1.0122, -0.0185, -1.0257, -1.0116,  0.7771,  0.2897, -2.1133]), 1)


[tensor([142]),
 tensor([8]),
 tensor([[ 0.3021, -0.6646, -0.1803, -1.9617,  0.3723, -3.1426, -0.0324]]),
 tensor([0])]

In [ ]:
class SuperModel(nn.Module):
    def __init__(self, num_batters, num_pitch_types, num_input_data_types, output_dim, batter_embedding=10, pitch_embedding=5, hidden_dim=64):
        super().__init__()
        self.batter_embedding = nn.Embedding(num_batters, batter_embedding)
        self.pitch_embedding = nn.Embedding(num_pitch_types, pitch_embedding)
        self.linear1 = nn.Linear(num_input_data_types, 32)
        self.linear2 = nn.Linear(32, 16)

        self.fc1 = nn.Linear(batter_embedding + pitch_embedding + 16, hidden_dim)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim, output_dim)
    def forward(self, batter_id, pitch_id, input_data):
        b = self.batter_embedding(batter_id)
        p = self.pitch_embedding(pitch_id) 
        i = self.linear1(input_data)
        i = self.relu(i)
        i = self.linear2(i)
        i = self.relu(i)
        x = torch.cat([b, p, i], dim=1)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

In [ ]:
model_0 = SuperModel(num_batters=NUM_BATTERS, num_pitch_types=NUM_PITCHES, num_input_data_types=7, output_dim=NUM_OUTCOMES)